In [1]:
import networkx as nx 
import numpy as np

from segk import segk
from utils import load_graph_classification_data, pyramid_match_kernel
from evaluation import evaluate_graph_classification

In [2]:
dataset = "MUTAG"
use_node_labels = False

nodes, edgelist, graph_indicator, node_labels, class_labels = load_graph_classification_data(dataset, use_node_labels)

n_graphs = max(graph_indicator.values())

E_segk_sp = segk(nodes, edgelist, radius=2, dim=20, kernel='shortest_path')
E_segk_wl = segk(nodes, edgelist, radius=2, dim=20, kernel='weisfeiler_lehman')

E_segk_sp -= np.min(E_segk_sp)
E_segk_wl -= np.min(E_segk_wl)

E_segk_sp /= np.max(E_segk_sp)
E_segk_wl /= np.max(E_segk_wl)

embeddings_segk_sp = list()
embeddings_segk_wl = list()
for i in range(n_graphs):
    embeddings_segk_sp.append(list())
    embeddings_segk_wl.append(list())
    
for i,node in enumerate(nodes):
    embeddings_segk_sp[graph_indicator[node]-1].append(E_segk_sp[i,:])
    embeddings_segk_wl[graph_indicator[node]-1].append(E_segk_wl[i,:])
    
embeddings_segk_sp = [np.vstack(l) for l in embeddings_segk_sp]
embeddings_segk_wl = [np.vstack(l) for l in embeddings_segk_wl]

K_segk_sp = pyramid_match_kernel(embeddings_segk_sp)
K_segk_wl = pyramid_match_kernel(embeddings_segk_wl)

In [3]:
print("Performance SEGK-SP")
print("-------------------")
evaluate_graph_classification(K_segk_sp, class_labels)

print()
print("Performance SEGK-WL")
print("-------------------")
evaluate_graph_classification(K_segk_wl, class_labels)

Performance SEGK-SP
-------------------

Mean performance on val set in 10 iterations: 0.901176
With standard deviation: 0.023500

Mean performance on test set in 10 iterations: 0.852193
With standard deviation: 0.023309

Performance SEGK-WL
-------------------

Mean performance on val set in 10 iterations: 0.915882
With standard deviation: 0.013682

Mean performance on test set in 10 iterations: 0.855673
With standard deviation: 0.014840
